In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install tensorflow-hub
!pip install numpy==1.16.1
!pip install keras==2.2.4
!wget -nc https://raw.githubusercontent.com/LanceNorskog/deep_meter_2/master/haiku_5.txt
!cut -f2 < haiku_5.txt | sort | uniq > haiku_5_short.txt
!wc -l haiku_5*.txt

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
File ‘haiku_5.txt’ already there; not retrieving.

   95631 haiku_5_short.txt
  673680 haiku_5.txt
  769311 total


In [2]:
!pip uninstall -qy git+https://github.com/LanceNorskog/deep_meter_2#egg=deepmeter
!pip install -q git+https://github.com/LanceNorskog/deep_meter_2#egg=deepmeter


In [3]:

from __future__ import print_function
import math
import pickle
import json
import os
import glob
import time

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import keras.backend as K
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, Sequential, load_model
from keras import layers 
from keras import metrics
from keras.preprocessing import text
from sklearn.model_selection import train_test_split

from cmu.syllables_cmu import syllables as word2sylls
from cmu.mappers import Decoder, trim_homynyms
from search.full import FullSearch
from cmu.topk import topk as get_top_k
from cmu.wordmap import Wordmap
from cmu.readhaiku import Reader
#from cmu.report import find_top_k_match, report
from keras_stuff.loss import sparse_categorical_crossentropy as scc
from keras_stuff.loss import sparse_categorical_crossentropy_temporal as scct
print(word2sylls['therefore'])

# number of total samples to use
max_data = 200000
# cut texts after this number of words
# number of output syllables in short haiku
max_features = 16000
# longest output sentence
max_len = 5
# longest input sentence
max_words = 10
# what you think
batch_size = 32
# do not output the same haiku twice
deduplicate_haiku=False
# emit output as input
duplicate_haiku=False

model_base="/content/gdrive/My Drive/Colab Notebooks/haiku_USE_5"
model_file=model_base + ".h5".format(int(time.time()))
print(model_file)


Using TensorFlow backend.


['DH EH R', 'F AO R']
/content/gdrive/My Drive/Colab Notebooks/haiku_USE_5.h5


In [4]:
!date
word2sylls = trim_homynyms(word2sylls)
decoder = Decoder(word2sylls)
syll2idx = decoder.syll2idx
idx2syll = decoder.idx2syll
num_sylls = len(idx2syll)

print(syll2idx['DH EH R'], idx2syll[1])
print('# features: ', len(idx2syll))

for i in range(decoder.wordoff):
    decoder.wordlist[i] = 'word{}'.format(i)
    decoder.wordlength[i] = 1
for i in range(decoder.sylloff):
    decoder.idx2syll[i] = 'syll{}'.format(i)

big_haiku_file = "haiku_5.txt"

wordmap = Wordmap(len(decoder.wordlist))
reader = Reader(word2sylls, decoder, wordmap)
(big_text, big_haiku, big_data) = reader.readfile(big_haiku_file)
haikuwordset = reader.haikuwordset
print('{} -> {} : {}'.format(big_text[0], big_haiku[0], big_data[0]))

print('Full length clauses: ', len(big_text))
print('Wordmap total entries: ', wordmap.count())
print('Wordmap length: ', wordmap.length())

Sun Jul 28 01:49:45 UTC 2019
2443 0
# features:  15098
office equipment -> office equipment : [ 211 2732 4968 6573 7383]
office equipment -> office equipment : [[ 211]
 [2732]
 [4968]
 [6573]
 [7383]]
Full length clauses:  40658
Wordmap total entries:  47775
Wordmap length:  210354


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [6]:
(x_train, x_test, y_train, y_test) = train_test_split(big_text, big_data)
x_train = x_train[0:(len(x_train) // batch_size) * batch_size]
y_train = y_train[0:(len(y_train) // batch_size) * batch_size]
x_test = x_test[0:(len(x_test) // batch_size) * batch_size]
y_test = y_test[0:(len(y_test) // batch_size) * batch_size]

print(x_test[0], str(y_test[0]))

def get_lstm(size, return_sequences=True):
    return layers.CuDNNLSTM(size, return_sequences=return_sequences)

#x_train = np.array(x_train)
#x_test = np.array(x_test)
#y_train = np.expand_dims(y_train, -1)
#y_test = np.expand_dims(y_test, -1)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(y_test[0][0])



some other people [[9827]
 [ 156]
 [2447]
 [8724]
 [8482]]
x_train shape: (30464,)
x_test shape: (10144,)
y_train shape: (30464, 5, 1)
y_test shape: (10144, 5, 1)
[9827]


In [0]:
# https://github.com/philipperemy/keras-snail-attention/blob/master/attention.py
# Do these Dense layers need activation tanh?
# https://www.d2l.ai/chapter_attention-mechanism/attention.html
# k, q have attention, v does not?
class AttentionBlock(layers.Layer):

    def __init__(self, dims, k_size, v_size, seq_len=None, **kwargs):
        self.k_size = k_size
        self.seq_len = seq_len
        self.v_size = v_size
        self.dims = dims
        self.sqrt_k = math.sqrt(k_size)
        self.keys_fc = None
        self.queries_fc = None
        self.values_fc = None
        super(AttentionBlock, self).__init__(**kwargs)

    def build(self, input_shape):
        # https://stackoverflow.com/questions/54194724/how-to-use-keras-layers-in-custom-keras-layer
        self.keys_fc = layers.Dense(self.k_size)
        self.keys_fc.build((None, self.dims))
        self._trainable_weights.extend(self.keys_fc.trainable_weights)

        self.queries_fc = layers.Dense(self.k_size)
        self.queries_fc.build((None, self.dims))
        self._trainable_weights.extend(self.queries_fc.trainable_weights)

        self.values_fc = layers.Dense(self.v_size)
        self.values_fc.build((None, self.dims))
        self._trainable_weights.extend(self.values_fc.trainable_weights)

    def call(self, inputs, **kwargs):
        # check that the implementation matches exactly py torch.
        keys = self.keys_fc(inputs)
        queries = self.queries_fc(inputs)
        values = self.values_fc(inputs)
        logits = K.batch_dot(queries, K.permute_dimensions(keys, (0, 2, 1)))
        mask = K.ones_like(logits) * np.triu((-np.inf) * np.ones(logits.shape.as_list()[1:]), k=1)
        logits = mask + logits
        probs = layers.Softmax(axis=-1)(logits / self.sqrt_k)
        read = K.batch_dot(probs, values)
        output = K.concatenate([inputs, read], axis=-1)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] += self.v_size
        return tuple(output_shape)



In [8]:
def sparse_categorical_accuracy(y_true, y_pred):
    # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
    if K.ndim(y_true) == K.ndim(y_pred):
        y_true = K.squeeze(y_true, -1)
    # convert dense predictions to labels
    y_pred_labels = K.argmax(y_pred, axis=-1)
    y_pred_labels = K.cast(y_pred_labels, K.floatx())
    return K.cast(K.equal(y_true, y_pred_labels), K.floatx())

def sparse_categorical_accuracy_per_sequence(y_true, y_pred):
    # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
    if K.ndim(y_true) == K.ndim(y_pred):
        y_true = K.squeeze(y_true, -1)
    # convert dense predictions to labels
    y_pred_labels = K.argmax(y_pred, axis=-1)
    y_pred_labels = K.cast(y_pred_labels, K.floatx())
    return K.min(K.cast(K.equal(y_true, y_pred_labels), K.floatx()), axis=-1)

def sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=5):
    original_shape = K.shape(y_true)
    y_true = K.reshape(y_true, (-1, K.shape(y_true)[-1]))
    y_pred = K.reshape(y_pred, (-1, K.shape(y_pred)[-1]))
    top_k = K.in_top_k(y_pred, K.cast(K.max(y_true, axis=-1), 'int32'), k)
    return K.reshape(top_k, original_shape[:-1])

def sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=5):
    original_shape = K.shape(y_true)
    y_true = K.reshape(y_true, (-1, K.shape(y_true)[-1]))
    y_pred = K.reshape(y_pred, (-1, K.shape(y_pred)[-1]))
    top_k = K.in_top_k(y_pred, K.cast(K.max(y_true, axis=-1), 'int32'), k)
    perfect = K.min(K.cast(top_k, 'int32'), axis=-1)
    return perfect #K.expand_dims(perfect, axis=-1)

def sparse(y_true, y_pred):
    return sparse_categorical_accuracy(y_true, y_pred)
def sparse1(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=1)
def perfect(y_true, y_pred):
    return sparse_categorical_accuracy_per_sequence(y_true, y_pred)
def perfect1(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=1)
def sparse5(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=5)
def perfect5(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=5)
def fscore(y_true, y_pred):
    recall = K.mean(sparse_categorical_accuracy(y_true, y_pred))
    precision = K.mean(sparse_categorical_accuracy_per_sequence(y_true, y_pred))
    return 2 * ((recall * precision)/(recall + precision))

def sparse_loss(y_true, y_pred):
    return scc(y_true, y_pred)

def perfect_loss(y_true, y_pred):
    return scct(y_true, y_pred, scale=1.0)

units_k=embed_size
units_v=embed_size
units_v=embed_size//3
units=512
dropout=0.4

metric_list = [sparse, perfect]
metric_names = ['sparse', 'perfect']


input_text = layers.Input(shape=(1,), dtype=tf.string)
x = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,), name='TF-Hub')(input_text)
if False:
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(unit_medium, activation='relu')(x)
if False:
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(unit_small, activation='relu')(x)
if False:
    x = layers.Dropout(dropout)(x)
    x = AttentionBlock(embed_size, k_size=units_k, v_size=units_v)(x)
x = layers.RepeatVector(max_len)(x)
if True:
    x = layers.Dropout(dropout)(x)
    x = AttentionBlock(embed_size, k_size=units_k, v_size=units_v)(x)
x = layers.Dropout(dropout)(x)
x = get_lstm(units, return_sequences=True)(x)
if True:
    x = layers.Dropout(dropout)(x)
    x = AttentionBlock(embed_size, k_size=units_k, v_size=units_v)(x)
x = layers.Dropout(dropout)(x)
output_layer = layers.Dense(max_features, activation='softmax')(x)


model = Model(inputs=[input_text], outputs=output_layer)
model.compile('adam', sparse_loss, metrics=metric_list)
model.summary()

# try using different optimizers and different optimizer configs

print('Train...')
history = None
use_saved_model=True
if not use_saved_model or not os.path.exists(model_file):
  with tf.Session() as session:
    K.manual_variable_initialization(False)
    model_file=model_base + ".h5".format(int(time.time()))
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())

    history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          callbacks=[EarlyStopping(monitor='val_perfect', mode='max', verbose=1, patience=10),
            ModelCheckpoint(model_file, monitor='val_perfect', save_best_only=True, save_weights_only=True, mode='max', verbose=0)],
          verbose=2,
          validation_data=[x_test, y_test])
    model.save_weights(model_file)


W0728 01:50:16.527220 140401553680256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 01:50:16.528907 140401553680256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 01:50:17.318040 140401553680256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0728 01:50:17.327586 140401553680256 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a futur

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
TF-Hub (Lambda)              (None, 512)               0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 512)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 512)            0         
_________________________________________________________________
attention_block_1 (Attention (None, 5, 682)            612522    
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 682)            0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 5, 512)            2449408   
__________

In [9]:

plt.figure()
if history != None:
  # summarize history for accuracy
  for m in metric_names:
      #plt.plot(history.history[m])
      plt.plot(history.history['val_' + m])
  plt.title('model accuracy')
  plt.xlabel('epoch')
  sname = []
  for m in metric_names:
      sname.append('{}={:01.3f}'.format(m, history.history['val_' + m][-1]))
  plt.legend(sname, loc='lower right')
  plt.show()

<Figure size 432x288 with 0 Axes>

In [10]:


x_short = big_text[0:500]
y_short = big_data[0:500]
predicts = None

with tf.Session() as session:
    #K.manual_variable_initialization(True)  
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    # need this?
    session.run(tf.tables_initializer())
    #model = load_model(model_file)  
    model.load_weights(model_file)  
    eval = model.evaluate(x_train, y_train)
    print('model.evaluate on short: ' ,model.metrics_names, eval)
    predicts = model.predict(x_short, batch_size=32)
    print('shape: {}'.format(predicts.shape))

print(len(predicts[0]))
print(len(predicts[0][0]))
print(predicts[0])

30464/30464 [==============================] - 25s 827us/step
model.evaluate on short:  ['loss', 'sparse', 'perfect'] [0.4242363538546096, 0.9026391802966094, 0.7501969537815126]
shape: (500, 5, 16000)
5
16000
[[2.2796884e-10 2.2763902e-10 2.2919523e-10 ... 2.2466705e-10
  2.2474850e-10 2.2584686e-10]
 [1.2497636e-13 1.2494919e-13 1.2988002e-13 ... 1.2551481e-13
  1.2927501e-13 1.2390165e-13]
 [5.9984590e-17 6.1312758e-17 6.0831438e-17 ... 5.8041556e-17
  5.9609811e-17 5.9504389e-17]
 [3.4098510e-19 3.3017403e-19 3.4410284e-19 ... 3.3824377e-19
  3.2904991e-19 3.2384470e-19]
 [3.3415643e-16 3.3163698e-16 3.2803723e-16 ... 3.4137027e-16
  3.2937893e-16 3.2102922e-16]]


In [0]:
def find_top_k_match(data, prediction, top_k=5):
        out = [-1] * len(data)
        for i in range(len(data)):
            topind = np.argsort(prediction[i])
            topind = topind[-top_k:]
            for j in range(top_k):
                #print(data[i][0], topind[j])
                if data[i][0] == topind[j]:
                    out[i] = topind[j]
        return out
    
def report(data, prediction):
    def match(data, prediction):
        assert len(data.shape) == 2
        assert len(prediction.shape) == 2
        good = 0
        top5 = 0
        count = 0
        for i in range(len(data)):
            topind = np.argsort(prediction[i])
            if data[i][0] == topind[-1]:
                good += 1
            topind = topind[-5:len(topind)]
            for j in range(5):
                if data[i][0] == topind[j]:
                    top5 += 1
                    break
            count += 1
        return (good, top5, count)

    _sparse = 0.0
    _perfect = 0.0
    _sparse5 = 0.0
    _perfect5 = 0.0
    _total = 0
    for n in range(len(data)):
        #print(len(short[n]))
        (good, top5, count) = match(data[n], predicts[n])
        if count == 0:
            continue
        _sparse += good/count
        _sparse5 += top5/count
        if good == count:
            _perfect += 1  
        if top5 == count:
            _perfect5 += 1
        _total += 1
    return {'sparse':_sparse/_total, 'perfect': _perfect/_total, 'sparse5': _sparse5/_total, 'perfect5': _perfect5/_total}


In [12]:
for i in range(len(y_short)):
    rep = report([y_short[i]], [predicts[i]])
    if rep['perfect5'] == 1.0:
        f = find_top_k_match(y_short[i], predicts[i], 5)
        print('[{}]: {} -> {}'.format(i, x_short[i], [decoder.idx2syll[j] for j in f]))

[0]: office equipment -> ['AO', 'F AH S', 'IH', 'K W IH P', 'M AH N T']
[1]: office equipment -> ['AO', 'F AH S', 'IH', 'K W IH P', 'M AH N T']


In [0]:
top_k=5

fs = FullSearch(top_k * top_k * top_k, 5, top_k)
def decodem(predict, top_k):
    global fs
    (top_vals, top_paths) = get_top_k(np.array([predict]), top_k=top_k)
    #print('top_paths: ' + str(top_paths))
    #print(top_paths.shape)
    fs.mainloop(top_paths[0])
    #print('score[0]: {}'.format(fs.scorevals[0]))
    #print('paths[0]: {}'.format(fs.scorepaths[0]))
    #print('score[-1]: {}'.format(fs.scorevals[-1]))
    #print('paths[-1]: {}'.format(fs.scorepaths[-1]))
    #print('min {}, max {}'.format(np.min(fs.scorevals), np.max(fs.scorevals)))
    morepaths = np.zeros(fs.scorepaths.shape, dtype='int32')
    for j in range(fs.scorepaths.shape[0]):
        #print('scorepaths[{}]: {}'.format(j, fs.scorepaths[j]))
        #print('predict.shape: ', predict.shape)
        #print('top_paths.shape: ', top_paths.shape)
        #print('top_paths[{}]: {}'.format(j, top_paths))
        #print('top_paths[{}][]: {}'.format(j, top_paths[0][np.arange(max_len), fs.scorepaths[j]]))
        morepaths[j] = top_paths[0][np.arange(max_len), fs.scorepaths[j]]
    #print('morepaths: ' + str(morepaths))
    encoded = decoder.get_sentences(morepaths)
    sentences = {}
    if len(encoded) > 0:
        #print(encoded)
        decoded = []
        for e1 in encoded:
            if len(e1) > 0 and len(e1[0]) > 0:
                dec = decoder.decode_sentences([e1])
                decoded.append(dec)
        for d1 in decoded:
            for d2 in d1:
                for d3 in d2:
                    for d4 in d3:
                        go = True
                        _lastidx = -1
                        for w in d4:
                            if not w in haikuwordset:
                                go = False
                            _idx = decoder.word2idx[w]
                            if _lastidx > 0:
                                if _lastidx == _idx or not wordmap.get(_lastidx, _idx):
                                    go = False
                                    #print('Fail: {},{} {},{}'.format(_lastidx, _idx, _lastword, w))
                            _lastidx = _idx
                            _lastword = w
                        if go:
                            key = ' '.join(d4)
                            sentences[key] = d4
                    #print('d3: ', d3)
                    #key = ' '.join(d3)
                    #sentences[key] = d3
    return sentences

# return N possible sentences with the fewest words
def short_sentences(sentences):
    out = {}
    for i in range(1, max_len + 1):
        for (k, v) in sentences.items():
            if len(v) == i:
                out[k] = v
        if len(out) > 0:
            return out

In [14]:
   
bigbatch = batch_size * 32
big_eval = big_text[0:bigbatch]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights(model_file)  
  biglen = len(x_test)
  for i in range(0, biglen, bigbatch):
      predicts = model.predict(big_eval[i:i + bigbatch], batch_size=bigbatch)
      for j in range(0, len(predicts)):
          #f = find_top_k_match(y_test[i + j], predicts[j], 5)
          #if np.min(f) > 0 and j == 0:
          #    print('{} -> {}'.format(x_test[i + j], [decoder.idx2syll[k] for k in f]))
          sentences = decodem(predicts[j], 5)
          if len(sentences) > 0:
              for s in short_sentences(sentences):
                    print('{} -> {}'.format(x_test[i + j], s))
              #print('{} -> {}'.format(x_test[i + j], sentences[0]))
              #for k in range(1, len(sentences)):
              #      print('. -> {}'.format(sentences[k]))
    

Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7fb1503e8cf8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


KeyboardInterrupt: ignored

In [0]:

(mini_vals, mini_preds) = get_top_k(np.array(predicts), top_k=top_k)
#print('top preds: ', mini_preds[0])
#print(mini_preds.shape)
total = 0
_go = []
for x in mini_preds[0]:
            _go.append(decoder.idx2syll[x[0]])
print('{} -> {}'.format(x_short[0], str(_go)))
for i in range(len(mini_preds)):
    fs = FullSearch(top_k * top_k * top_k, 5, top_k)
    fs.mainloop(mini_preds[i])
    #print('score[0]: {}'.format(fs.scorevals[0]))
    #print('paths[0]: {}'.format(fs.scorepaths[0]))
    #print('score[-1]: {}'.format(fs.scorevals[-1]))
    #print('paths[-1]: {}'.format(fs.scorepaths[-1]))
    #print('min {}, max {}'.format(np.min(fs.scorevals), np.max(fs.scorevals)))
    print('{} -> {}'.format(x_short[i], [decoder.idx2syll[x] for x in fs.scorepaths[0]]))
    morepaths = np.zeros(fs.scorepaths.shape, dtype='int32')
    print(mini_preds[i].shape)
    print(morepaths.shape)
    for j in range(fs.scorepaths.shape[0]):
        #print(fs.scorepaths[j])
        #z = mini_preds[i][np.arange(max_len), fs.scorepaths[j]]
        #print(z)
        morepaths[j] = mini_preds[i][np.arange(max_len), fs.scorepaths[j]]
    #print(morepaths[0])
    encoded = decoder.get_sentences(morepaths)
    if i == 0:
        print('encoded[0]: ', encoded[0])
    #d = []
    # for x in encoded:
    #   for y in x:
    #        if len(y) > 0:
    #           d.append(y[0])  
    #d = np.array(d)
    d = encoded
    if len(d) > 0:
        print('encoded sentences: ', d)
        print(x_short[i], ':')
        #print(mini_preds[0])
        total += 1
        decoded = decoder.decode_sentences(encoded)
        #print('len(decoded): ', len(decoded))
        ##print('len(decoded[0]): ', len(decoded[0]))
        #print('len(decoded[0][0]): ', len(decoded[0][0]))
        #print('len(decoded[0][0][0]): ', len(decoded[0][0][0]))
        #print('decoded[0][0][0]: ', decoded[0][0][0])
        sentences = {}
        for d1 in decoded:
            for d2 in d1:
                for d3 in d2:
                    print('d3: ', d3)
                    break
                    key = ' '.join(d3)
                    sentences[key] = d3
                    break
                     
        print('[{}]  -> {}', i,list(sentences.keys())[0:10])
    break
print('Total decoded: {}'.format(total))

In [0]:
with tf.Session() as session:
  #K.manual_variable_initialization(True)  
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  #model = load_model(model_file)  
  model.load_weights(model_file)  
  eval_small = model.evaluate(big_haiku, big_data)
  print('model.evaluate on haiku clauses: ' ,model.metrics_names, eval_small)
  print('history: ', history)
  eval_big = model.evaluate(big_text, big_data)
  print('model.evaluate on long clauses: ' ,model.metrics_names, eval_big)
  print('history: ', history)
  biglen = len(big_text)
  #for i in range(0, len(big_text), batch_size):
  #  predicts = model.predict(big_text[i:i + batch_size], batch_size=batch_size)
  #  print('shape: {}'.format(predicts.shape))


In [0]:
metric_list = [sparse, perfect, sparse5, perfect5]
metric_names = ['sparse', 'perfect', 'sparse5', 'perfect5']

model = Model(inputs=[input_text], outputs=output_layer)
model.compile('adam', 'sparse_categorical_crossentropy', metrics=metric_list)

bigbatch = batch_size * 32
big_text = np.array(big_text)
big_haiku = np.array(big_haiku)
text5arr = []
haiku5mean = None
with tf.Session() as session:
  #K.manual_variable_initialization(True)  
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  #model = load_model(model_file)  
  model.load_weights(model_file)  
  predicts = model.predict(big_haiku[0: bigbatch], batch_size=bigbatch)
  rep = report(big_data[0: bigbatch], predicts)
  print("short {}".format(rep))
  haiku5mean = rep['perfect5']
  biglen = len(big_text)
  for i in range(0, biglen, bigbatch):
    predicts = model.predict(big_text[i:i + bigbatch], batch_size=bigbatch)
    rep = report(big_data[i:i + bigbatch], predicts)
    print("[{}] {}".format(i, rep))
    text5arr.append(rep['perfect5'])

text5mean = np.mean(np.array(text5arr))

In [0]:
val5arr = []
with tf.Session() as session:
  #K.manual_variable_initialization(True)  
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  #model = load_model(model_file)  
  model.load_weights(model_file)  
  biglen = len(x_test)
  for i in range(0, biglen, bigbatch):
    predicts = model.predict(x_test[i:i + bigbatch], batch_size=bigbatch)
    rep = report(y_test[i:i + bigbatch], predicts)
    print("[{}] {}".format(i, rep))
    val5arr.append(rep['perfect5'])

val5mean = np.mean(np.array(val5arr))

In [0]:
print('Perfect5 for all haiku lines: {}, all mscoco lines: {}, validation mscoco: {}'.format(haiku5mean, text5mean, val5mean))